In [4]:
from textwrap import dedent
import numpy as np
from quantecon.lqcontrol import LQ
from quantecon.quadsums import var_quadratic_sum
from numpy import dot, log, sqrt, identity, hstack, vstack, trace
from scipy.linalg import solve, inv, det
from quantecon.matrix_eqn import solve_discrete_lyapunov

class RBLQ(object):
    r"""
    provides methods for analysing infinite horizon robust LQ control
    problems of the form
    
        ..math::
        
            min_{u_t} sum_t beta^t {x_t' R x_t +u_t' R u_t }
            
        subject to
        ..math::
            
            x_{t+1}=A x_t +B u_t +C w_{t+1}
        
    and with model misspecification parameter theta.
    
    parameters
    --------
    Q: array_like(float, ndim=2)
        the cost(payoff) matrix for the controls. see above for more.
        Q should be k x k and symmetric and positive definite
    R: array_like(float, ndim=2)
        the cost(payoff) matrix for the state. see above for more.
        R should be n x n and symmetric and non-negative definite.
    A: array_like(float, ndim=2)
        the matrix that corresponds with the state in the state space
        system. A should be n x n
    B: array_like(float, ndim=2)
        the matrix that corresponds with the control in the state space
        system. B should be n x k
    C: array_like(float, ndim=2)
        the matrix that corresponds with the random process in the
        state space system. C should be n x j
    beta: scalar(float)
        the discount factor in the robust control problem
    theta: scalar(float)
        the robustness factor in the robust control problem
    
    attributes
    ---------
    Q, R, A, B, C, beta, theta: see parameters
    k, n, j: scalar(int)
        the dimensions of the matrices
        
    """
    
    def __init__(self, Q, R, A, B, C, beta, theta):
        
        #==make sure all matrices, can be treated as 2d arrays==#
        A, B, C, Q, R =list(map(np.atleast_2d, (A, B, C, Q, R)))
        self.A, self.B, self.C, self.Q, self.R=A, B, C, Q, R
        #==record dimensions==#
        self.k=self.Q.shape[0]
        self.n=self.R.shape[0]
        self.j=self.C.shape[1]
        #==remaining parameters==#
        self.beta, self.theta=beta, theta
        
    def __repr__(self):
        return self.__str__()
    
    def __str__(self):
        m="""\
        robust linear quadratic control system
            -beta (discount parameter): {b}
            -theta (robustness factor): {th}
            -n (number of state variables): {n}
            -k (number of control variables): {k}
            -j (number of shocks): {j}
        """
        return dedent(m.format(b=self.beta, n=self.n, k=self.k, j=self.j,
                              th=self.theta))
    def d_operator(self, P):
        r"""
        the D operator, mapping P into
        
            ..math::
                
                D(P):=P+PC(theta I -C'PC)^{-1} C'P
                
        parameters
        ----------
        P: array_like(float, ndim=2)
            a matrix that should be n x n
            
        returns
        ---------
        dP: array_like(float, ndim=2)
            the matrix P after applying the D operator
            
        """
        C, theta=self.C, self.theta
        I=np.identity(self.j)
        S1=dot(P, C)
        S2=dot(C.T, S1)
        
        dP=P+dot(S1, solve(theta*I-S2, S1.T))
        
        return dP
    
    def b_operator(self, P):
        r"""
        the B operator mapping P into
        
        
            ..math::
                B(P):=R=beta^2 A'PB(Q+beta B'PB)^{-1}B'PA +beta A'PA
                
            and also returning
            
            ..math::
                F:=(Q+beta B'PB)^{-1} beta B'PA
                
        parameters
        ----------
        P: array_like(float, ndim=2)
            a matrix that should be n x n
        
        returns
        ----------
        F: array_like(float, ndim=2)
            the F matrix as defined above
        new_P: array_like(float, ndim=2)
            the matrix P after applying the B operator
            
        """
        A, B, Q, R, beta=self.A, self.B, self.Q, self.R, self.beta
        S1=Q+beta